In [14]:
import time
from selenium import webdriver
import Queue as Q
import pandas as pd
import numpy as np
import pyautogui
import os

username='rsk05'
password='swaminathan'
world=99
driverLocation='/home/rsk/Downloads/chromedriver'



In [4]:
#Game login

driver = webdriver.Chrome(driverLocation)

driver.get('https://en'+str(world)+'.tribalwars.net/game.php')

uname = driver.find_element_by_name('username')
uname.send_keys(username)

pword = driver.find_element_by_name('password')
pword.send_keys(password)

login= driver.find_element_by_class_name('btn-login')
login.click()

time.sleep(3)
driver.get('https://www.tribalwars.net/en-dk/page/play/en'+str(world))

In [11]:
def getUnits():
    
    driver.get('https://en'+str(world)+'.tribalwars.net/game.php?screen=overview')
    
    unitsData = driver.find_element_by_id("show_units")
    
    t=unitsData.text
    t = t.split("\n")
    t = t[1:-1]
    
    unitsInfo={}
    
    unitNames = {'Spear':'spear','Swordsmen':'sword','Axemen':'axe','Archers':'archer','Scouts':'spy','Light':'light'}
    
    for types in t:
        
        types = types.split(" ")
        if types[1] in unitNames:
            unitsInfo[unitNames[types[1]]] = int(types[0].replace('.',''))
        else:
            unitsInfo[str(types[1])] = int(types[0].replace('.',''))
        
    return(unitsInfo)

In [5]:
def sendAttack(coords,units,lastAttackTime):
    """
    Sends attack to a given coordinate location with specified troops. 
    
    The function returns 0 if there are not enough troops to send
    
    Returns 1 if the last attack to this village was sent less than an hour ago
    
    Else returns the current time when the attack was sent
    
    """
    
    currentURL = driver.current_url
    
    if currentURL[-8:] == 'overview':
        newURL = currentURL[:-8]+'place'
        driver.get(newURL)
    else:
        driver.get('https://en'+str(world)+'.tribalwars.net/game.php?screen=place')
    
    coordsInput = driver.find_element_by_name('input')
    coordsInput.send_keys(str(coords[0])+'|'+str(coords[1]))
    
    times = driver.find_element_by_id('serverTime').text
    date = driver.find_element_by_id('serverDate').text
    timeDate = times+' '+date
    currentTime = pd.to_datetime(timeDate,format='%H:%M:%S %d/%m/%Y')
    
    #check time gap of 1 hr with previous attack
    if (currentTime-lastAttackTime).total_seconds()<=3600.0:
        return(1)
    
    for unitType in units:
    
        availUnits = int(driver.find_element_by_id('units_entry_all_'+unitType).text[1:-1])
        
        #check if required units are available
        if availUnits < units[unitType]:
            print("Insufficient troops. Attack to ({}|{}) not sent".format(coords[0],coords[1]))
            return(0)    
        
        unitInput = driver.find_element_by_name(unitType)
        unitInput.send_keys(units[unitType])
    
    
    attackButton = driver.find_element_by_name('attack')
    attackButton.click()
    
    sendButton = driver.find_element_by_name('submit')
    sendButton.click()
    
    return(timeDate)
    
    

In [ ]:
def createDataFile(fileName):
    
    headerText = 'x,y,key,points,lastAttacked,prevFarm,prevScout,template,fullReturns,totalFarm,totalAttacks,wall,dist,order,close'
    
    
    

In [3]:
def checkReports(numPages=4):
    
    villageList=[]
    idList = []
    i=0


    while(True):

        driver.get('https://en'+str(world)+'.tribalwars.net/game.php?screen=report&mode=attack&from='+str(12*i))
        table = driver.find_element_by_id('report_list')
        reportList=table.text.split('\n')[1:-1]

        villagelist,idlist,stopInd = getReportID()

        villageList += villagelist
        idList += idlist

        #if we have reached old reports, then stop
        if stopInd==1:
            break

        # if no of pages exceed limit, then stop
        if i==(numPages-1):
            break

        i+=1
        
    finalList = zip(idList, villageList)
    
    fileName = 'barbsList.csv'
    
    
    if os.path.isfile(fileName):
        data = pd.read_csv(fileName)
    else:
        createDataFile(fileName)
    
    
    
    for i in range(len(finalList)):
        driver.get('https://en'+str(world)+'.tribalwars.net/game.php?screen=report&mode=attack&view='+str(finalList[i][0]))

        result = driver.find_element_by_id("attack_results")

        haul = int(result.text.split("\n")[0].split(" ")[-1].split("/")[0].replace('.',''))
        haulLimit = int(result.text.split("\n")[0].split(" ")[-1].split("/")[1].replace('.',''))
        
        fullReturns = 1 if haul==haulLimit else 0

        wall = getWall(driver.find_element_by_id('attack_spy_buildings_right').text.split("\n")[1:])

        scoutHaul = getScoutHaul(driver.find_element_by_id("attack_spy_resources").text[19:])
        
        if fullReturns==1 and scoutHaul>haulLimit:
            order = 1
        else:
            order = 0
        

        villageKey = finalList[i][1]
        
        print([villageKey,haul,fullReturns,scoutHaul,order])


        data.loc[data.key==villageKey,'prevFarm'] = haul
        data.loc[data.key==villageKey,'prevScout'] = scoutHaul
        data.loc[data.key==villageKey,'wall'] = wall
        data.loc[data.key==villageKey,'totalFarm'] = data.loc[data.key==villageKey,'totalFarm'] + haul
        data.loc[data.key==villageKey,'totalAttacks'] = data.loc[data.key==villageKey,'totalAttacks'] + 1
        data.loc[data.key==villageKey,'fullReturns'] = fullReturns
        data.loc[data.key==villageKey,'order'] = order
        
    
    data.to_csv('barbsList99.csv',index=False,index_label=False)

In [18]:
import os
fileName = 'barbsList99.csv'

os.path.isfile(fileName) 

True

In [4]:

attackTemps = {
    'ssTemp': {'spear':20,'spy':1,'axe':3},
    'ssTemp2': {'spear':10,'sword':6,'axe':2},
    'lcTemp3' : {'light':3,'spy':1},
    'lcTemp4' : {'light':4,'spy':1}    
}


def getWall(buildList):
    level = 0
    for i in range(len(buildList)):
        if "Wall" in buildList[i]:
            level = int(buildList[i].split(" ")[1])
    return(level)

def getScoutHaul(spyReport):
    scout= 0
    if spyReport!="none":
        scout = np.sum([int(t.replace('.','')) for t in spyReport.split(" ")])
    return(scout)
    

def getReportID():
    
    table = driver.find_element_by_id('report_list')
    reportList=table.text.split('\n')[1:-1] 
    reports=table.find_elements_by_tag_name("tr")[1:-1]
    stopIndicator=0
    
    idList = []
    villageList = []

    for i in range(len(reportList)):


        isNew = 1 if '(new)' in reportList[i] else 0
        if isNew ==0:
            stopIndicator=1
            break
            
        ind = reportList[i].find('|')
        coords = [int(reportList[i][ind-3:ind]) , int(reportList[i][ind+1:ind+4]) ]
        village = coords[0]*1000+coords[1]
        
        if village not in villageList:
            
            villageList.append(village)

            reportID = reports[i].find_elements_by_tag_name("td")[0].find_element_by_tag_name("input").get_attribute('name')[3:]
            idList.append(reportID)
            
    
    return([villageList,idList,stopIndicator])
        

In [5]:
def startFarm(fileName,close=False):
    
    checkReports()

    data = pd.read_csv(fileName)

    data['lastAttacked'] = data['lastAttacked'].apply(lambda x : pd.to_datetime(x,format='%Y-%m-%d %H:%M:%S'))
    
    dataClose = data[data.close==1]
    dataFar = data[data.close==0]
    
    # Farming close barbs

    dataClose = dataClose.sort_values(['order','lastAttacked'],ascending=[0,1])

    for i in range(len(dataClose)):
        sample = dataClose.iloc[i]
        coords = [sample['x'],sample['y']]
        key = sample['key']
        lastAttackTime = sample['lastAttacked']
        template = sample['template']
        
        
        
        try:
            out = sendAttack(coords, attackTemps[template],lastAttackTime)
        except:
            out=0
        
        if out==0:
            break
        elif out==1:
            continue
        else:
            print("Village {} | Order : {} Last Attack : {} Close : {}".format(key, sample['order'],lastAttackTime,sample['close']))
            dataClose.loc[dataClose.key==key,'lastAttacked'] = pd.to_datetime(out,format='%H:%M:%S %d/%m/%Y')

            
            
    #Farming far barbs
    
    dataFar = dataFar.sort_values(['order','lastAttacked'],ascending=[0,1])

    for i in range(len(dataFar)):
        sample = dataFar.iloc[i]
        coords = [sample['x'],sample['y']]
        key = sample['key']
        lastAttackTime = sample['lastAttacked']
        template = sample['template']
        
        
        
        try:
            out = sendAttack(coords, attackTemps[template],lastAttackTime)
        except:
            out=0
        
        if out==0:
            break
        elif out==1:
            continue
        else:
            print("Village {} | Order : {} Last Attack : {} Close : {}".format(key, sample['order'],lastAttackTime,sample['close']))
            dataFar.loc[dataFar.key==key,'lastAttacked'] = pd.to_datetime(out,format='%H:%M:%S %d/%m/%Y')
    
    data = pd.concat([dataClose,dataFar],axis=0)
    data.to_csv(fileName,index=False,index_label=False)

    
def addFarm(filename, checkCoords,close=0):
    
    data = pd.read_csv(filename)
    if checkCoords[0]*1000+checkCoords[1] in data.key.values:
        print("Farm exists in dataset")
    else:
        print("Adding Farm to dataset")


        row={'x':checkCoords[0],'y':checkCoords[1],'key':checkCoords[0]*1000+checkCoords[1],'points':0,
             'lastAttacked':pd.to_datetime('20:24:27 10/05/2018', format='%H:%M:%S %d/%m/%Y'),
             'prevFarm':0,'prevScout':0,
             'template':'ssTemp','fullReturns':0,'totalFarm':0,'totalAttacks':0,'wall':0,'dist':0,'order':0,
             'close':0}
        
        data = data.append(row,ignore_index=True)
        data.to_csv(filename,index=False,index_label=False)

In [42]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

In [72]:
for i in range(4):
    driver.execute_script("window.open('');")
    driver.switch_to_window(driver.window_handles[-1])

    driver.get('https://en99.tribalwars.net/game.php?village=36880&screen=place&mode=command')

#     unitInput = driver.find_element_by_name('spear').send_keys(12)
    unitInput = driver.find_element_by_name('axe').send_keys(1)
    coordsInput = driver.find_element_by_name('input').send_keys('602|637')
    driver.find_element_by_name('attack').click()
    

In [69]:
pyautogui.position()

(174, 67)

In [71]:
driver.switch_to_window(driver.window_handles[0])

In [23]:
Keys.


In [4]:
driver.get('http://google.com')
ActionChains(driver).key_down(Keys.CONTROL).key_down('t').key_up(Keys.CONTROL).key_up('t').perform()

In [9]:

ActionChains(driver).key_down(Keys.CONTROL).key_down(Keys.TAB).key_up(Keys.CONTROL).key_up(Keys.TAB).perform()

In [131]:

ActionChains(driver).key_up(Keys.CONTROL).perform()        

In [150]:
ActionChains(driver).key_down(Keys.CONTROL).send_keys(Keys.TAB).key_up(Keys.CONTROL).perform()        

In [38]:
e = driver.find_element_by_id("content_value")

In [43]:
f = e.find_element_by_xpath('//*[@class="vis modemenu"]/tbody/tr/td')

In [45]:
f.click()

## Farming

In [19]:
startFarm('barbsList99.csv')

[615385, 411, 0, 0, 0]
[608378, 38, 0, 3, 0]
[597385, 240, 1, 1167, 1]
[598386, 240, 1, 247, 1]
[597384, 240, 1, 1504, 1]
[610372, 215, 0, 0, 0]
[598383, 900, 0, 3, 0]
Village 604388 | Order : 0 Last Attack : 2018-05-14 05:54:49 Close : 0
Insufficient troops. Attack to (601|377) not sent


In [112]:
checkReports()

[608374, 163, 0, 0, 0]
[614379, 43, 0, 0, 0]
[609374, 0, 0, 0, 0]


## Results

In [74]:
data = pd.read_csv('barbsList99.csv')

In [51]:
data = data.sort_values(['order','lastAttacked'],ascending=[0,1])
data.head()

,x,y,key,points,lastAttacked,prevFarm,prevScout,template,fullReturns,totalFarm,totalAttacks,wall,dist,order
40,608,378,608378,0,2018-05-13 06:50:06,0,0,ssTemp,0,0,0,0,0,0
41,607,381,607381,0,2018-05-13 06:50:09,373,0,ssTemp,0,373,1,0,0,0
42,613,379,613379,0,2018-05-13 06:50:12,443,3,ssTemp,0,443,1,0,0,0
43,611,383,611383,0,2018-05-13 06:50:14,149,0,ssTemp,0,149,1,0,0,0
44,609,383,609383,0,2018-05-13 06:50:17,288,0,ssTemp,0,288,1,0,0,0


In [52]:
data = data.sort_values(['order','lastAttacked'],ascending=[0,1])
data.head()

,x,y,key,points,prevFarm,prevScout,fullReturns,totalFarm,totalAttacks,wall,dist,order
count,46.000000,46.000000,46.000000,46.0,46.000000,46.000000,46.0,46.000000,46.000000,46.000000,46.0,46.0
mean,605.108696,380.543478,605489.239130,0.0,223.673913,244.760870,0.0,228.891304,0.956522,0.021739,0.0,0.0
std,5.669522,5.348755,5668.776812,0.0,277.631735,497.356978,0.0,309.371997,0.294884,0.147442,0.0,0.0
min,597.000000,372.000000,597384.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0
25%,600.000000,375.500000,600378.000000,0.0,82.250000,0.000000,0.0,82.250000,1.000000,0.000000,0.0,0.0
50%,604.500000,381.000000,604889.000000,0.0,240.000000,3.000000,0.0,240.000000,1.000000,0.000000,0.0,0.0
75%,609.000000,385.000000,609380.750000,0.0,240.000000,218.500000,0.0,240.000000,1.000000,0.000000,0.0,0.0
max,615.000000,391.000000,615385.000000,0.0,1853.000000,2057.000000,0.0,2093.000000,2.000000,1.000000,0.0,0.0


In [75]:
data[data.close==0].sort_values(['order','lastAttacked'],ascending=[0,1])

,x,y,key,points,lastAttacked,prevFarm,prevScout,template,fullReturns,totalFarm,totalAttacks,wall,dist,order,close
19,607,386,607386,0,2018-05-13 14:37:01,160,263,lcTemp3,1,160,1,1,0,1,0
20,607,391,607391,0,2018-05-13 14:37:03,240,339,lcTemp3,1,240,1,0,0,1,0
21,598,380,598380,0,2018-05-13 14:37:09,240,674,lcTemp3,1,240,1,0,0,1,0
22,599,381,599381,0,2018-05-13 14:37:14,240,626,lcTemp3,1,240,1,0,0,1,0
23,599,383,599383,0,2018-05-13 14:37:17,240,2628,lcTemp3,1,240,1,0,0,1,0
32,598,374,598374,0,2018-05-13 11:18:55,212,3,lcTemp3,0,212,1,0,0,0,0
33,601,373,601373,0,2018-05-13 11:19:01,0,0,lcTemp3,0,0,0,0,0,0,0
34,602,373,602373,0,2018-05-13 11:19:04,0,0,lcTemp3,0,0,0,0,0,0,0
35,603,373,603373,0,2018-05-13 11:19:06,0,0,lcTemp3,0,0,0,0,0,0,0
36,604,389,604389,0,2018-05-13 11:19:21,0,0,lcTemp3,0,0,0,0,0,0,0


In [99]:
e=data.iloc[2]

In [88]:
t1 = pd.to_datetime('20:24:27 10/05/2018', format='%H:%M:%S %d/%m/%Y')
t2 = pd.to_datetime('21:24:27 10/05/2018', format='%H:%M:%S %d/%m/%Y')

In [94]:
(t2-t1).total_seconds()

3600.0

In [101]:
e['lastAttacked']

'2018-05-13 14:36:54'

In [29]:
a = pd.DataFrame(np.array([[34,24,45,42,53],[12,45,13,67,23]]).T,columns=['a','b'])

In [31]:
a['c'] = a.apply(lambda x : x['a']+x['b'], axis=1)

In [38]:
a

,a,b,c
0,34,12,46
1,24,45,69
2,45,13,58
3,42,67,109
4,53,23,76


In [44]:
a = a.sort_values('a')

In [45]:
a

,a,b,c
1,24,45,69
0,34,12,46
3,42,67,109
2,45,13,58
4,53,23,76


In [58]:
a1 = a[a.a>40]
a2 = a[a.a<40]

In [61]:
a1.b=0

/usr/local/lib/python2.7/dist-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [64]:
c = pd.concat([a1,a2],axis=0)
c

,a,b,c
3,42,0,109
2,45,0,58
4,53,0,76
1,24,45,69
0,34,12,46


In [63]:
a

,a,b,c
1,24,45,69
0,34,12,46
3,42,67,109
2,45,13,58
4,53,23,76


In [ ]:

data = pd.read_csv('farBarbsList99.csv')

data['lastAttacked'] = data['lastAttacked'].apply(lambda x : pd.to_datetime(x,format='%Y-%m-%d %H:%M:%S'))

data = data.sort_values('lastAttacked')


for i in range(len(data)):
    coords = [data.iloc[i]['x'],data.iloc[i]['y']]
    key = data.iloc[i]['key']
    
    temp = data.iloc[i]['template']
    
    if temp=='lcTemp3':
        out = sendAttack(coords,lcTemp3)
    elif temp=='lcTemp4':
        out = sendAttack(coords,lcTemp4)
        
    if out==0:
        break
    else:
        data.loc[data.key==key,'lastAttacked'] = pd.to_datetime(out,format='%H:%M:%S %d/%m/%Y')
        print(data.loc[data.key==key,'lastAttacked'])
        

data.to_csv('farBarbsList.csv',index=False,index_label=False)
        

In [109]:
farBarbsList=[[488,481],
             [488,477],
             [490,477],
             [489,480],
             [490,481],
             [490,482],
             [491,481],
             [491,483],
             [491,484],
             [485,483],
             [482,479],
             [483,475],
             [484,473],
             [488,486],
             [491,485],
             [493,484],
             [492,487],
             [491,488],
             [493,488],
             [492,489]]

for i in range(len(farBarbsList)):
    farBarbsList[i] = (0,farBarbsList[i])

In [31]:
closeBarbsList=[(0,[610,381]),
               (0,[608,378]),
               (0,[607,381]),
               (0,[613,379]),
                (0,[611,383]),
                (0,[609,383]),
                (0,[608,375]),
                (0,[609,374]),
                (0,[608,374]),
                (0,[608,373]),
                (0,[606,374]),
                (0,[612,373]),
                (0,[614,381]),
                (0,[615,381]),
                (0,[614,382])
               ]

In [179]:
a = np.array([x[1][0] for x in farBarbsList])
b = np.array([x[1][1] for x in farBarbsList])

df = pd.DataFrame(np.array([a,b]).T,columns = ['x','y'])

df['key'] = df.apply(lambda x : str(x['x'])+str(x['y']),axis=1)
df['points'] = df.apply( lambda x : 0, axis=1)
df['lastAttacked'] = df.apply( lambda x : pd.to_datetime('20:24:27 10/05/2018', format='%H:%M:%S %d/%m/%Y'), axis=1)
df['prevFarm'] = df.apply(lambda x: 0, axis=1)
df['prevScout'] = df.apply(lambda x: 0, axis=1)
df['template'] = df.apply(lambda x:'lcTemp3', axis=1)
df['fullReturns'] = df.apply(lambda x : 0, axis=1)
df['totalFarm'] = df.apply(lambda x : 0, axis=1)
df['totalAttacks'] = df.apply(lambda x : 0, axis=1)
df['wall'] = df.apply(lambda x : 0, axis=1)
df['dist'] = df.apply(lambda x : 0, axis=1)
df['order'] = df.apply(lambda x : 0, axis=1)
df['close'] = df.apply(lambda x : 0, axis=1)

df.to_csv('farBarbsList99.csv',index=False,index_label=False)


In [39]:
# Add a new column to data file
#Make sure to modify addFarm function on adding new column
data = pd.read_csv('barbsList99.csv')
data['order'] = data.apply(lambda x : 0,axis=1)
data.to_csv('barbsList99.csv',index=False,index_label=False)

In [67]:
len(closeBarbsList)

15

In [178]:
farBarbsList=[[603,373],
             [602,373],
             [601,373],
             [600,373],
              [598,374],
              [601,377],
              [600,377],
              [603,379],
              [598,380],
              [599,381],
              [600,381],
              [598,383],
              [599,383],
              [597,384],
              [598,384],
              [597,385],
              [599,386],
              [600,386],
              [602,386],
              [598,386],
              [607,386],
              [606,387],
              [604,386],
              [604,388],
              [605,389],
              [604,389],
              [607,391]]

for i in range(len(farBarbsList)):
    farBarbsList[i] = (0,farBarbsList[i])
        

In [228]:
data1 = pd.read_csv('closeBarbsList99.csv')
data2 = pd.read_csv('farBarbsList99.csv')

data = pd.concat([data1,data2],axis=0)

data.to_csv('barbsList99.csv',index=False,index_label=False)